# Biomarker_Extraction


In [ ]:
import pandas as pd
import numpy as np
import os

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})
pd.options.display.float_format = '{:,.5f}'.format
pd.set_option("display.max_rows", None)

## Merge t-test data

* `R_code > t-test_2group.R` 로 생성한 파일 사용



## Merge Survival data

* `R_code > survival_analysis.R` 로 생성한 파일 사용
* 생존기간에 따른 파일 생성 (OS/DSS/DFI/PFI)

In [ ]:
# R로 만든 파일 불러오기
base_path='New_Data/forR/survival_summary/'
list_ = os.listdir(base_path)

In [ ]:
# 파일 병합
df_all=pd.DataFrame(columns=['category', 'SLC', 'method', 'cutoff', 'OS_error', 'OS_p_value', 'OS_HR', 'OS_AUC', 'OS_sensitivity', 'OS_specificity', 'OS_graph_name_jpg', 'DSS_error', 'DSS_p_value', 'DSS_HR', 'DSS_AUC', 'DSS_sensitivity', 'DSS_specificity', 'DSS_graph_name_jpg', 'DFI_error', 'DFI_p_value', 'DFI_HR', 'DFI_AUC', 'DFI_sensitivity', 'DFI_specificity', 'DFI_graph_name_jpg', 'PFI_error', 'PFI_p_value', 'PFI_HR', 'PFI_AUC', 'PFI_sensitivity', 'PFI_specificity', 'PFI_graph_name_jpg'])
for li in list_:
    df=pd.read_csv(base_path+li,engine='python')
    df.drop(columns=['Unnamed: 0'],inplace=True)
    df_all=pd.concat([df_all,df],axis=0)

In [ ]:
# 파일 복사
df_os = df_all.copy()
df_dss = df_all.copy()
df_dfi = df_all.copy()
df_pfi = df_all.copy()

# 필요없는 column drop
df_os.dropna(subset=['OS_p_value', 'OS_HR'],inplace=True)
df_dss.dropna(subset=['DSS_p_value', 'DSS_HR'], inplace=True)
df_dfi.dropna(subset=['DFI_p_value', 'DFI_HR'], inplace=True)
df_pfi.dropna(subset=['PFI_p_value', 'PFI_HR'], inplace=True)

# p-value <=0.05 & HR >=1인 경우만 추출
df_os_biomarker = df_os[(df_os['OS_p_value']<=0.05) & (df_os['OS_HR']>=1)]
df_dss_biomarker = df_dss[((df_dss['DSS_p_value']<=0.05) & (df_dss['DSS_HR']>=1))]
df_dfi_biomarker = df_dfi[((df_dfi['DFI_p_value']<=0.05) & (df_dfi['DFI_HR']>=1))]
df_pfi_biomarker = df_pfi[((df_pfi['PFI_p_value']<=0.05) & (df_pfi['PFI_HR']>=1))]

In [ ]:
# 파일 내보내기 -> 선택사항
# df_os_biomarker.to_csv('New_Data/survival_new/os_biomarker.csv', index=False)
# df_dss_biomarker.to_csv('New_Data/survival_new/dss_biomarker.csv', index=False)
# df_dfi_biomarker.to_csv('New_Data/survival_new/dfi_biomarker.csv', index=False)
# df_pfi_biomarker.to_csv('New_Data/survival_new/pfi_biomarker.csv', index=False)

## Make Final Data

* p-value <= 0.05 & HR >= 1

In [ ]:
df_bio=df_os[(df_os['DSS_p_value']<=0.05) & (df_os['DSS_HR']>=1) &
      (df_os['DFI_p_value']<=0.05) & (df_os['DFI_HR']>=1) &
      (df_os['PFI_p_value']<=0.05) & (df_os['PFI_HR']>=1) ]

df=pd.merge(df_t,df_bio,how='inner',on=['category','SLC'])
# df['category'].value_counts() # 암종별 SLC 갯수 확인

In [ ]:
# # 내보내기 -> 선택사항
# df.to_csv('New_Data/SLC_biomarker_innerjoin.csv', index=False)

In [ ]:
# 각 암종별 biomarker
slc_cat = []
for i in category:
    slc = []
    df_ = df[df['category'] == i]

    if len(df_) != 0 : 
        slc = df_[df_['category']== i]['SLC'].unique()

    slc_cat.append((i, slc))

In [ ]:
# SLC-암종리스트 리스트 형식 데이터
arr = []
for i in slcs:
    cate_slc = df[df['SLC']==i].drop_duplicates('category')
    cancers = cate_slc['category'].values
    arr.append((i, cancers.tolist()))

In [ ]:
# DataFrame으로 만들기
df_SLC = pd.DataFrame(arr, columns=['SLC', 'cancer'])
df_SLC['count'] = df_SLC['cancer'].apply(lambda x:len(x))
df_SLC.sort_values(by='count', inplace=True, ascending=False)
df_SLC.reset_index(drop=True, inplace=True)

In [ ]:
# # 내보내기 -> 선택사항
# df_SLC.to_csv('New_Data/slc218_cancerlist.csv', index=False)

In [ ]:
# 최종 파일 생성
df_mat = pd.read_csv('New_Data/TCGA_GTEX_matched_label.csv')

# bio-marker 선정
df_col = ['sample','TCGA_GTEX_main_category', 'cancer', 'label', 'label_GTEx_100']
slc_list = df['SLC'].unique().tolist()
col_list = slc_list + df_col

df_fin = df_mat[col_list]

In [ ]:
# 파일 내보내기
df_fin.to_csv('./New_Data/TCGA_GTEX_SLC_103.csv',index=False)

* p-value <= 0.01 & HR >= 1

In [ ]:
df_bio=df_os[(df_os['DSS_p_value']<=0.01) & (df_os['DSS_HR']>=1) &
      (df_os['DFI_p_value']<=0.01) & (df_os['DFI_HR']>=1) &
      (df_os['PFI_p_value']<=0.01) & (df_os['PFI_HR']>=1) ]

df=pd.merge(df_t,df_bio,how='inner',on=['category','SLC'])
# df['category'].value_counts() # 암종별 SLC 갯수 확인

In [ ]:
# 내보내기
df.to_csv('New_Data/SLC_biomarker_innerjoin_001.csv', index=False)

In [ ]:
# SLC-암종 리스트 형식
arr = []
for i in slcs:
    cate_slc = df[df['SLC']==i].drop_duplicates('category')
    cancers = cate_slc['category'].values
    arr.append((i, cancers.tolist()))

In [ ]:
# 리스트 형식 데이터를 DataFrame으로 만들기
df_SLC = pd.DataFrame(arr, columns=['SLC', 'cancer'])
df_SLC['count'] = df_SLC['cancer'].apply(lambda x:len(x))
df_SLC.sort_values(by='count', inplace=True, ascending=False)
df_SLC.reset_index(drop=True, inplace=True)

In [ ]:
# # csv 파일로 내보내기 -> 선택사항
# df_SLC.to_csv('New_Data/slc103_cancerlist.csv', index=False)

In [ ]:
# 최종 파일 생성
df_mat = pd.read_csv('New_Data/TCGA_GTEX_matched_label.csv')

# bio-marker 선정
df_col = ['sample','TCGA_GTEX_main_category', 'cancer', 'label', 'label_GTEx_100']
slc_list = df['SLC'].unique().tolist()
col_list = slc_list + df_col

df_fin = df_mat[col_list]

In [ ]:
# 파일 내보내기
df_fin.to_csv('./New_Data/TCGA_GTEX_SLC_.csv',index=False)